#### This project is trying to maximize the equation: Y = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + w6x6 (x=input, w=weight)

#### Genetic Algorithm Flowchart:
<img src="https://miro.medium.com/max/968/1*plKxNWoJ8u19UplCbeN9SQ.png">

The idea of maximizing such equation seems simple. The positive input is to be multiplied by the largest possible positive number and the negative number is to be multiplied by the smallest possible negative number. But the idea we are looking to implement is how to make GA do that its own in order to know that it is better to use positive weight with positive inputs and negative weights with negative inputs.

Create a list of the 6 inputs and a variable to hold the number of weights

In [1]:
# Inputs of the equation.
equation_inputs = [4,-2,3.5,5,-11,-4.7]

# Number of the weights we are looking to optimize.
num_weights = 6

#### Initial Population
Define the initial population. Based on the number of weights, each chromosome (solution or individual) in the population will definitely have 6 genes, one gene for each weight. There is no fixed value for that and we can select the value that fits well with our problem. But we could leave it generic so that it can be changed in the code. Next, we create a variable that holds the number of solutions per population, another to hold the size of the population, and finally, a variable that holds the actual initial population:

In [6]:
import numpy
sol_per_pop = 8

# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
print("Population: ",pop_size)
print("8 chromosomes and each one has 6 genes, one for each weight")

#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)
print("\nNew Population: ",new_population)

Population:  (8, 6)
8 chromosomes and each one has 6 genes, one for each weight

New Population:  [[-1.99883211 -0.40378765 -2.61393079 -3.873453    3.03698178  3.56653698]
 [-3.735449   -0.55194895  3.23997124  2.77657898 -2.39080936 -3.33471065]
 [-1.73435959 -1.1564164   1.62835674  0.27307984 -2.53746535  0.0092781 ]
 [ 2.92449847  0.1673516  -1.13007533 -0.79825109 -3.56754324 -3.18645004]
 [-1.85953133 -3.4263049  -2.22529784 -1.75383969 -0.52653171 -3.68885847]
 [ 3.4746645  -3.95729271 -2.55040149  0.29520757  0.29304696  1.09725548]
 [ 0.25803914  3.3353221   0.10710668  3.13063863  2.10414787  3.56305608]
 [ 0.27290384  1.3614849   1.10818136  3.8724172  -1.1999307   0.49383548]]


Note that it is generated randomly and thus it will definitely change when get run again.

#### Pre-defined GA module from the article:

In [7]:
import numpy

def cal_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

It will run through 5 generation. Step for each generation:
<ol>
    <li>The first step is to <b>find the fitness value</b> of each solution within the population using the <code>cal_pop_fitness</code> function. The fitness function accepts both the equation inputs values (x1 to x6) in addition to the population. The fitness value is calculated as the sum of product (SOP) between each input and its corresponding gene (weight) according to our function. According to the number of solutions per population, there will be a number of SOPs. As we previously set the number of solutions to 8 in the variable named sol_per_pop, there will be 8 SOPs</li>
    <li><b>Select the best of them as parents</b> in the mating pool according to the next function <code>select_mating_pool</code>. It accepts the population, the fitness values, and the number of parents needed. It returns the parents selected.</li>
    <li>Use such selected parents for mating in order to <b>generate the offspring</b>. The mating starts with the crossover operation according to the <code>crossover</code> function. This function accepts the parents and the offspring size. It uses the offspring size to know the number of offspring to produce from such parents.</li>
    <li>Next <b>mutation</b>, to the results of the crossover stored in the offspring variable using the <code>mutation</code> function. Such function accepts the crossover offspring and returns them after applying uniform mutation.</li>
    <li>It is preferred to keep the previous best solutions (parents) in the new population. In the worst case when all the new offspring are worse than such parents, we will continue using such parents. As a result, we guarantee that the new generation will at least preserve the previous good results and will not go worse. The new population will have its first 4 solutions from the previous parents. The last 4 solutions come from the offspring created after applying crossover and mutation</li>
    <li>Pick the best solution from the new population</li>
</ol>

In [14]:
num_generations = 5
num_parents_mating = 4

for generation in range(num_generations):
    print("\nGeneration : ", generation)
    # Measing the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(equation_inputs, new_population)
    print("Fitness values: ",fitness)

    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, 
                                      num_parents_mating)
    print("Selected Parents: ",parents)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], num_weights))
    print("Crossover result: ",offspring_crossover)
    
    # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover)
    print("Mutation result: ",offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration.
    print("Best result : ", numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))


Generation :  0
Fitness values:  [157.03884009 149.4391733  148.2709876  146.75572273 143.88068107
 145.87461441 144.08211719 161.71746616]
Selected Parents:  [[  2.92449847   0.1673516   -1.13007533   2.77657898 -11.34121856
   -3.33471065]
 [  2.92449847   0.1673516   -1.13007533   2.77657898 -10.91588892
   -3.33471065]
 [  2.92449847   0.1673516   -1.13007533   2.77657898 -10.22501012
   -3.33471065]
 [  2.92449847   0.1673516   -1.13007533   2.77657898 -10.11881142
   -3.33471065]]
Crossover result:  [[  2.92449847   0.1673516   -1.13007533   2.77657898 -10.91588892
   -3.33471065]
 [  2.92449847   0.1673516   -1.13007533   2.77657898 -10.22501012
   -3.33471065]
 [  2.92449847   0.1673516   -1.13007533   2.77657898 -10.11881142
   -3.33471065]
 [  2.92449847   0.1673516   -1.13007533   2.77657898 -11.34121856
   -3.33471065]]
Mutation result:  [[  2.92449847   0.1673516   -1.13007533   2.77657898 -11.4433854
   -3.33471065]
 [  2.92449847   0.1673516   -1.13007533   2.77657898  

#### Getting the best solution after iterating finishing all generations.
At first, the fitness is calculated for each solution in the final generation. Then return the index of that solution corresponding to the best fitness. 

In [16]:
fitness = cal_pop_fitness(equation_inputs, new_population)
best_match_idx = numpy.where(fitness == numpy.max(fitness))

print("\nBest solution : ", new_population[best_match_idx, :])
print("\nBest solution fitness : ", fitness[best_match_idx])


Best solution :  [[[  2.92449847   0.1673516   -1.13007533   2.77657898 -13.60187339
    -3.33471065]]]

Best solution fitness :  [186.58466928]
